## BERT MODEL
change your runtime to gpu


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 54.2 MB/s eta 0:00:00


In [ ]:
from transformers import BertTokenizer, BertModel
import torch

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [ ]:
def get_embeddings(text):
    tokens = tokenizer.tokenize(text)

    # Truncate or pad the tokens to match the maximum sequence length
    max_seq_length = 128
    if len(tokens) > max_seq_length - 2:
        tokens = tokens[:max_seq_length - 2]
    else:
        tokens = tokens + ['[PAD]'] * (max_seq_length - 2 - len(tokens))

    # Add special tokens and convert tokens to input IDs
    tokens = ['[CLS]'] + tokens + ['[SEP]']
    input_ids = torch.tensor([tokenizer.convert_tokens_to_ids(tokens)]).to(device)

    with torch.no_grad():
        outputs = model(input_ids)
        embeddings = outputs.last_hidden_state

    return embeddings

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
#The file is shared in whatsapp group too
csv_path = '/content/drive/MyDrive/main_df_filtered_lem_stem2.csv'
df = pd.read_csv(csv_path)

In [ ]:
import math

data = []
for i, row in df.iterrows():
  issue = row['text']
  cmnts =row['cmts_text']
  t=''
  if not (isinstance(cmnts, float) and math.isnan(cmnts)):
    t = ' '.join([issue,cmnts]) 
  else:
    if (isinstance(issue, float) and math.isnan(issue)):
      t= ' '
    else:
      t = issue
  data.append(t)

Get Embeddings

In [ ]:
import time
import math
start_time = time.time()
tensor_list = []
for i,issue in enumerate(data):
  if not (isinstance(issue, float) and math.isnan(issue)):
    tensor_list.append(get_embeddings(issue))

    if i%100 == 0:
      elapsed_time = time.time() - start_time
      print(f"Iteration {i}: Elapsed time - {elapsed_time:.2f} seconds")

Iteration 0: Elapsed time - 3.07 seconds
Iteration 100: Elapsed time - 4.56 seconds
Iteration 200: Elapsed time - 6.06 seconds
Iteration 300: Elapsed time - 7.59 seconds
Iteration 400: Elapsed time - 9.81 seconds
Iteration 500: Elapsed time - 12.15 seconds
Iteration 600: Elapsed time - 14.42 seconds
Iteration 700: Elapsed time - 17.19 seconds
Iteration 800: Elapsed time - 18.61 seconds
Iteration 900: Elapsed time - 20.11 seconds
Iteration 1000: Elapsed time - 21.74 seconds
Iteration 1100: Elapsed time - 23.33 seconds
Iteration 1200: Elapsed time - 25.01 seconds
Iteration 1300: Elapsed time - 27.09 seconds
Iteration 1400: Elapsed time - 32.40 seconds
Iteration 1500: Elapsed time - 33.88 seconds
Iteration 1600: Elapsed time - 35.48 seconds
Iteration 1700: Elapsed time - 37.09 seconds
Iteration 1800: Elapsed time - 38.73 seconds
Iteration 1900: Elapsed time - 40.35 seconds
Iteration 2000: Elapsed time - 41.96 seconds
Iteration 2100: Elapsed time - 44.53 seconds
Iteration 2200: Elapsed tim

Save the tensor_list in batches

In [ ]:
import torch
import pickle

# Your tensor list

# Set the batch size
batch_size = 1000

# Determine the number of batches
num_batches = len(tensor_list) // batch_size + 1

# Save the tensors in batches
for i in range(num_batches):
    start_idx = i * batch_size
    end_idx = (i + 1) * batch_size
    batch_tensors = tensor_list[start_idx:end_idx]

    # Save the batch of tensors to a file
    filename = f'tensor_batch_{i}.pt'
    torch.save(batch_tensors, filename)

# Save the list metadata (e.g., total length) for reference
metadata = {'length': len(tensor_list), 'batch_size': batch_size, 'num_batches': num_batches}
with open('metadata.pkl', 'wb') as f:
    pickle.dump(metadata, f)

Load tensor_list

In [ ]:
import torch
import pickle

# Load the metadata
with open('metadata.pkl', 'rb') as f:
    metadata = pickle.load(f)

length = metadata['length']
batch_size = metadata['batch_size']
num_batches = metadata['num_batches']

# Initialize an empty list to store the tensors
tensor_list = []

# Load and concatenate the tensors from each batch
for i in range(num_batches):
    filename = f'tensor_batch_{i}.pt'
    batch_tensors = torch.load(filename)
    tensor_list.extend(batch_tensors)

# Ensure the final list has the correct length
tensor_list = tensor_list[:length]

# Now you have a single list of tensors containing all the tensors from the batches

In this part we are going to preprocess the open issue

In [ ]:
import pandas as pd
import re
from string import punctuation
from nltk.corpus import stopwords
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')

stop_words = list(punctuation) + ["'s","'m","n't","'re","-","'ll",'...'] + stopwords.words('english')

def preprocess(line):
    ## Replace all code blocks with the token CODE
    pattern = re.compile(r'```[^```]*?(```|$)', re.MULTILINE|re.DOTALL)
    line = re.sub(pattern,'CODE',line)
    pattern = re.compile(r'`[^`]*?`', re.MULTILINE|re.DOTALL)
    line = re.sub(pattern,'CODE',line)
    pattern = re.compile(r'(^|\n)>.*')
    line = re.sub(pattern,'\nREFERENCE',line)
    pattern = re.compile(r'On [\d]+ \w+ [\d]+ at [\d]+:[\d]+, \w+ \w+ <.*?> wrote:')
    line = re.sub(pattern,'REFERENCE',line)
    pattern = re.compile(r'On \w+, \w+ \d+, \d+, \d+:\d+ (A|P)M \w+ \w+ .*? wrote:')
    line = re.sub(pattern,'REFERENCE',line)
    pattern = re.compile(r'([\n\t\s]*?REFERENCE)+',re.MULTILINE|re.DOTALL)
    line = re.sub(pattern,'\nREFERENCE',line)
    pattern = re.compile(r'\[[^\]]+\]\([^)]+\)')
    line = re.sub(pattern, ' URL ', line)
    
    
    tokens = word_tokenize(line)

    #stopword
    stop_tokens = [word for word in tokens if word.lower() not in stop_words]
        
    #lem
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in stop_tokens]
    
    
    #stem
    stemmer = PorterStemmer()
    stemmed_tokens = [stemmer.stem(token) for token in lemmatized_tokens]
    
    result = ' '.join(stemmed_tokens)
    
    
    return result

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


We are taking the open Issue from our closed issues for now

In [ ]:
preprocessed_issue_indices = [x for x in range(0,10)] + [x for x in range(8555,8565)] + [x for x in range(12000,12010)]

In [ ]:
preprocessed_open_issue = data[2]   # for example we are taking issue number 2 here
preprocessed_open_issue_list = data[0:10] + data[8555:8565] + data [12000:12010]   # issue list

open_issue_embedding = get_embeddings(preprocessed_open_issue)
open_issue_list_embedding = list(map(get_embeddings, preprocessed_open_issue_list))

In [ ]:
preprocessed_open_issue_list

['feat(platform-server): Expose XHR client used platform-server/http ## PR Checklist Please check PR fulfills following requirements: [x] commit message follows guidelines: https://github.com/angular/angular/blob/master/CONTRIBUTING.md#commit [x] Tests changes added (for bug fixes features) [x] Docs added updated (for bug fixes features) ## PR Type kind change PR introduce? <!-- Please check one applies PR using "x". --> Bugfix [x] Feature Code style update (formatting, local variables) Refactoring (no functional changes, api changes) Build related changes CI related changes Documentation content changes angular.io application infrastructure changes Other... Please describe: ## current behavior? <!-- Please describe current behavior modifying, link relevant issue. --> wish attach cookies HTTP requests SSR. CODE URL I\'m using following https://github.com/angular/angular/issues/15730#issuecomment-291578814 CODE import xhr2 \'xhr2\'; xhr2.prototype._restrictedHeaders.cookie false; CODE E

In [ ]:
open_issue_list_embedding

[tensor([[[-0.3526, -0.0639,  0.1808,  ..., -0.2547, -0.3290,  0.9007],
          [ 0.1464, -0.2379,  0.4241,  ...,  0.1122,  0.7036,  0.8519],
          [-1.2018, -0.4240,  0.2021,  ..., -0.2608, -0.0795,  0.3604],
          ...,
          [ 0.6284, -0.3771, -0.1819,  ..., -0.4702, -0.6744,  0.5349],
          [ 0.2381,  0.0885, -0.1530,  ..., -0.3172, -0.2495, -0.2150],
          [ 0.5114,  0.2623, -0.1861,  ...,  0.2143, -0.6298, -0.2292]]],
        device='cuda:0'),
 tensor([[[-0.3636,  0.0201,  0.2765,  ..., -0.3760, -0.1981,  0.7026],
          [-1.1339, -0.8832,  0.4945,  ..., -0.3896,  0.1612,  1.2021],
          [-0.3803, -0.2950,  0.6244,  ...,  0.2999, -0.4039,  0.2389],
          ...,
          [-0.2282, -0.1040,  0.1761,  ..., -0.3417, -0.7327,  0.9332],
          [ 0.0760,  0.2414,  0.4052,  ..., -0.5356, -0.6301, -0.3161],
          [ 0.4781,  0.2310, -0.2293,  ...,  0.1346, -0.5690, -0.2022]]],
        device='cuda:0'),
 tensor([[[-4.7443e-01, -7.2234e-01, -2.9480e-02, 

Now we calculate the cosine similarity

In [ ]:
import torch.nn.functional as F

# Assuming you have two tensors `embedding1` and `embedding2` containing embeddings
# Calculate the cosine similarity between the two tensors
def sim_cal(embedding):
  #print(embedding)
  embedding1 = embedding.squeeze(0)
  embedding2 = open_issue_embedding.squeeze(0)
  embedding1 = embedding1.view(-1)
  embedding2 = embedding2.view(-1)
  similarity = F.cosine_similarity(embedding1, embedding2, dim=0)
  return similarity

similarities = list(map(sim_cal,tensor_list))


# Print the cosine similarity values

In [ ]:
import torch.nn.functional as F

# Assuming you have two tensors `embedding1` and `embedding2` containing embeddings
# Calculate the cosine similarity between the two tensors
def sim_cal_custom_issue(embedding, open_issue_embedding_custom):
  #print(embedding)
  embedding1 = embedding.squeeze(0)
  embedding2 = open_issue_embedding_custom.squeeze(0)
  embedding1 = embedding1.view(-1)
  embedding2 = embedding2.view(-1)
  similarity = F.cosine_similarity(embedding1, embedding2, dim=0)
  return similarity

# similarities = list(map(sim_cal,tensor_list))

In [ ]:
similarity_lists = []
for open_issue_embedding in open_issue_list_embedding:
  # similarities = list(map(sim_cal_custom_issue,open_issue_embedding,tensor_list))
  similarities = [sim_cal_custom_issue(closed_issue_embedding, open_issue_embedding) for closed_issue_embedding in tensor_list]
  similarity_lists.append(similarities)

In [ ]:
len(similarity_lists)

30

Sorting the similarities

In [ ]:
# Define a custom key function to extract the values from tensors
def get_tensor_value(tensor):
    return tensor.item()

# Sort the tensor list in decreasing order and retrieve the indices
sorted_indices = [i for i, _ in sorted(enumerate(similarities), key=lambda x: get_tensor_value(x[1]), reverse=True)]
sorted_similarities = [similarities[i] for i in sorted_indices]

In [ ]:
top_list = sorted_indices[:30] #take top 20 similar issues


# to make it in the form of fusion function's input
bert_list = []
for i,value in enumerate(top_list):
  bert_list.append((i+1,value))

In [ ]:
# Sorting methods for list of open issues
def get_tensor_value(tensor):
    return tensor.item()

def sort_by_similarity(single_issue_similarity):
  # Sort the tensor list in decreasing order and retrieve the indices
  sorted_indices = [i for i, _ in sorted(enumerate(single_issue_similarity), key=lambda x: get_tensor_value(x[1]), reverse=True)]
  sorted_similarities = [single_issue_similarity[i] for i in sorted_indices]
  return sorted_indices

sorted_similarity_lists = list(map(sort_by_similarity, similarity_lists))

In [ ]:
sorted_similarity_lists[4][0:19]

[4,
 966,
 4423,
 1520,
 131,
 954,
 4950,
 4596,
 5866,
 1965,
 96,
 84,
 1274,
 1297,
 1730,
 5793,
 3030,
 86,
 4194]

### Word2Vec

In [ ]:
import multiprocessing
import pandas as pd
from gensim.models import Word2Vec
from time import time
import nltk
from typing import List
from typing import Union
import numpy as np
from numpy.linalg import norm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

csv_path = '/content/drive/MyDrive/main_df_filtered_lem_stem2.csv'
df = pd.read_csv(csv_path)

In [ ]:
df = pd.read_csv ('main_df_filtered_lem_stem.csv')

FileNotFoundError: ignored

In [ ]:
print(len(df["text"]))
nltk.download('punkt')

12588


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
combined_text = df["text"] + df["cmts_text"]

#Replace NaN with ""
for i in range(len(combined_text)):
  if type(combined_text[i]) != str:
    combined_text[i] = ""

print(combined_text)

0        feat(platform-server): Expose XHR client used ...
1        ng-conf 2020 ## PR Checklist Please check PR f...
2        build: consolidate @angular-devkit/build-angul...
3        refactor(router): added type pathMatch pathMat...
4        perf: add js-web-frameworks benchmark (the pul...
                               ...                        
12583                                                     
12584    fix(runtime-core): ensure custom events emitte...
12585                                                     
12586    fix: Apply Ref Transform Named Exports Applies...
12587    fix(v-model): fix v-model generate ref assignm...
Length: 12588, dtype: object


In [ ]:
processed_dataset = []
comment_text = []
issue_text = []

for i in range(len(combined_text)):
  if type(df["text"][i]) != str:
    df["text"][i] = ""

  if type(df["cmts_text"][i]) != str:
    df["cmts_text"][i] = ""

  tokenized_issue_text = nltk.word_tokenize(df["text"][i])
  tokenized_comment_text = nltk.word_tokenize(df["cmts_text"][i])
  tokenized_combined_text = nltk.word_tokenize(combined_text[i])
  
  processed_dataset.append(tokenized_combined_text)
  issue_text.append(tokenized_issue_text)
  comment_text.append(tokenized_comment_text)
      
print(len(processed_dataset))

<ipython-input-7-31cf26469902>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["cmts_text"][i] = ""
<ipython-input-7-31cf26469902>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["text"][i] = ""


12588


In [ ]:
print(len(issue_text), len(comment_text))

12588 12588


In [ ]:
cores = multiprocessing.cpu_count()
print(f"The number of available cores is {cores}")

word2vec_model = Word2Vec(min_count = 0, window=5, vector_size=128,
                          sample= 0.001,
                          alpha=0.025,
                          negative=5,
                          workers=cores - 1)

w2v_start_time = time()

word2vec_model.build_vocab(corpus_iterable=processed_dataset, progress_per=128)

print('Time to build vocab: {} mins'.format(round((time() - w2v_start_time) / 60, 2)))
print(f"The num of sentences are: {word2vec_model.corpus_count}")
print(f"The num of words are: {word2vec_model.corpus_total_words}")

w2v_start_time = time()
word2vec_model.train(processed_dataset, total_examples=word2vec_model.corpus_count, epochs=100, report_delay=1)
print('Time to train the model: {} mins'.format(round((time() - w2v_start_time) / 60, 2)))

The number of available cores is 2
Time to build vocab: 0.05 mins
The num of sentences are: 12588
The num of words are: 4025737
Time to train the model: 13.45 mins


In [ ]:
word2vec_model.save("w2v_weights")

In [ ]:
word2vec_model = Word2Vec.load("w2v_weights")

In [ ]:
def text_mean_embedder(issue_text: List[str], comment_text: List[str], model: Word2Vec):
    if model:
        model_type = type(model)
        issue_vectors = []
        comment_vectors = []

        if model_type == Word2Vec:
            return model.wv.get_mean_vector(keys=issue_text, ignore_missing=True), model.wv.get_mean_vector(keys=comment_text, ignore_missing=True)

        # Getting none here means
        issue_vectors_avg = np.sum(a=issue_vectors, axis=0)/len(issue_vectors)
        comment_vectors_avg = np.sum(a=comment_vectors, axis=0)/len(comment_vectors)

        return issue_vectors_avg, comment_vectors_avg
    else:
        raise Exception("Empty model specified for text-embedding")

In [ ]:
def score(issue_text: List[str], comment_text: List[str], model: Word2Vec):
    # Calls embedding function
    issue_text_vector, comment_text_vector = text_mean_embedder(issue_text=issue_text, comment_text=comment_text, model=model)
    print("issue vector: ", len(issue_text_vector), "comment vector: ", len(comment_text_vector))
    return np.dot(issue_text_vector, comment_text_vector) / (norm(issue_text_vector) * norm(comment_text_vector))

In [ ]:
w2v_cos_sim_ranks = []

count = 0

for i in df["issue_number"]:
    issue_id = int(i)
   
    issue_text_value = issue_text[2] + comment_text[2]
    comment_text_value = comment_text[count] + issue_text[count]

    # query_vector, doc_vector = w2v_text_embedder(query_text_value=query_text_value, doc_text_value=doc_text_value, w2v_model=word2vec_model)
    if (len(issue_text_value) > 0 and len(comment_text_value) > 0):
        w2v_cos_sim = score(issue_text=issue_text_value, comment_text=comment_text_value, model=word2vec_model)
        doc_cosinesim_relevance_tuple = (count, w2v_cos_sim)
        w2v_cos_sim_ranks.append(doc_cosinesim_relevance_tuple)

    else:
        print("missed")
    count += 1

Streaming output truncated to the last 5000 lines.
issue vector:  128 comment vector:  128
issue vector:  128 comment vector:  128
issue vector:  128 comment vector:  128
issue vector:  128 comment vector:  128
issue vector:  128 comment vector:  128
issue vector:  128 comment vector:  128
issue vector:  128 comment vector:  128
issue vector:  128 comment vector:  128
issue vector:  128 comment vector:  128
issue vector:  128 comment vector:  128
issue vector:  128 comment vector:  128
issue vector:  128 comment vector:  128
issue vector:  128 comment vector:  128
issue vector:  128 comment vector:  128
issue vector:  128 comment vector:  128
issue vector:  128 comment vector:  128
issue vector:  128 comment vector:  128
issue vector:  128 comment vector:  128
issue vector:  128 comment vector:  128
issue vector:  128 comment vector:  128
issue vector:  128 comment vector:  128
issue vector:  128 comment vector:  128
issue vector:  128 comment vector:  128
issue vector:  128 comment ve

<ipython-input-14-caa4cd115286>:5: RuntimeWarning: invalid value encountered in float_scalars
  return np.dot(issue_text_vector, comment_text_vector) / (norm(issue_text_vector) * norm(comment_text_vector))


comment vector:  128
issue vector:  128 comment vector:  128
issue vector:  128 comment vector:  128
issue vector:  128 comment vector:  128
issue vector:  128 comment vector:  128
issue vector:  128 comment vector:  128
issue vector:  128 comment vector:  128
issue vector:  128 comment vector:  128
issue vector:  128 comment vector:  128
issue vector:  128 comment vector:  128
issue vector:  128 comment vector:  128
issue vector:  128 comment vector:  128
issue vector:  128 comment vector:  128
issue vector:  128 comment vector:  128
issue vector:  128 comment vector:  128
issue vector:  128 comment vector:  128
issue vector:  128 comment vector:  128
issue vector:  128 comment vector:  128
issue vector:  128 comment vector:  128
issue vector:  128 comment vector:  128
issue vector:  128 comment vector:  128
issue vector:  128 comment vector:  128
issue vector:  128 comment vector:  128
issue vector:  128 comment vector:  128
issue vector:  128 comment vector:  128
issue vector:  128 

In [ ]:
w2v_cos_sim_ranks.sort(key=lambda x: x[1], reverse=True)

In [ ]:
word2vec_top_20 = []
for i in range(20):
  print(w2v_cos_sim_ranks[i][0])
  word2vec_top_20.append((i+1, w2v_cos_sim_ranks[i][0]))

2
1026
234
5595
4340
2357
369
7636
5606
3968
6779
5767
6955
615
4451
6642
2814
5951
10
4014


In [ ]:
list2 = [(1, 2),
 (2, 2467),
 (3, 4911),
 (4, 4627),
 (5, 1825),
 (6, 7160),
 (7, 4536),
 (8, 3481),
 (9, 4760),
 (10, 4866),
 (11, 3817),
 (12, 538),
 (13, 5606),
 (14, 10093),
 (15, 2669),
 (16, 2446),
 (17, 5556),
 (18, 4354),
 (19, 1037),
 (20, 4738)]

In [ ]:
print(list2)
print(word2vec_top_20)

[(1, 2), (2, 2467), (3, 4911), (4, 4627), (5, 1825), (6, 7160), (7, 4536), (8, 3481), (9, 4760), (10, 4866), (11, 3817), (12, 538), (13, 5606), (14, 10093), (15, 2669), (16, 2446), (17, 5556), (18, 4354), (19, 1037), (20, 4738)]
[(1, 2), (2, 1026), (3, 234), (4, 5595), (5, 4340), (6, 2357), (7, 369), (8, 7636), (9, 5606), (10, 3968), (11, 6779), (12, 5767), (13, 6955), (14, 615), (15, 4451), (16, 6642), (17, 2814), (18, 5951), (19, 10), (20, 4014)]


In [ ]:
#Data Fusion
def rank_position_fusion(list1, list2):
    ranks = {}
    for i, item in enumerate(list1):
        rank, id_ = item[0], item[1]
        ranks[id_] = 1 / (rank)
    for i, item in enumerate(list2):
        rank, id_ = item[0], item[1]
        if id_ in ranks:
            ranks[id_] += 1 / (rank)
        else:
            ranks[id_] = 1 / (rank)
    sorted_ranks = sorted(ranks.items(), key=lambda x: x[1], reverse=True)
    fused_results = []
    for i, (id_, rank) in enumerate(sorted_ranks):
        fused_results.append([id_, i + 1])

    return fused_results


In [ ]:
l1 = [value for _,value in word2vec_top_20]
l2 = [value for _,value in list2]
set(l1) & set(l2)

{2, 5606}

In [ ]:
rank_position_fusion(list2,word2vec_top_20)

[[2, 1],
 [2467, 2],
 [1026, 3],
 [4911, 4],
 [234, 5],
 [4627, 6],
 [5595, 7],
 [1825, 8],
 [4340, 9],
 [5606, 10],
 [7160, 11],
 [2357, 12],
 [4536, 13],
 [369, 14],
 [3481, 15],
 [7636, 16],
 [4760, 17],
 [4866, 18],
 [3968, 19],
 [3817, 20],
 [6779, 21],
 [538, 22],
 [5767, 23],
 [6955, 24],
 [10093, 25],
 [615, 26],
 [2669, 27],
 [4451, 28],
 [2446, 29],
 [6642, 30],
 [5556, 31],
 [2814, 32],
 [4354, 33],
 [5951, 34],
 [1037, 35],
 [10, 36],
 [4738, 37],
 [4014, 38]]

Things below here are for experiment one. Initially I was going to write a separate python script for our Experiment 1 but since everything is already present here why export and import separately every time

In [ ]:
!pip install mosspy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# IMPORTS
import mosspy
import pandas as pd
from urllib.request import urlopen
import re

In [ ]:
# CONSTANTS
# replace as needed from
# umair - 346974895
# umair - 669838402
userid=907267466
# umair - 204215945
# umut - 907267466
# vahid - 940716647

In [ ]:
def moss_get_comparison_report_url(language, file1, file2):
    m = mosspy.Moss(user_id=userid, language=language)
    m.addFile(file_path=file1)
    m.addFile(file_path=file2)
    url = m.send(lambda file_path, display_name: print('*', end='', flush=True))
    print()
    print ("Report Url: " + url)
    return url


In [ ]:
def get_patch_url_from_issue_id(issue_id):
    if issue_id in df.index:
        patch_url = df["patch_url"][issue_id]
        # print(patch_url)
        return patch_url

In [ ]:
def get_file_from_url(url):
    with urlopen(url=url) as file:
        filedata = file.read().decode()
        return filedata

In [ ]:
def get_patch_file_from_issue_id(issue_id):
    if issue_id in df.index.values:
        url = get_patch_url_from_issue_id(issue_id=issue_id)
        # print(url)
        # print(type(url))
        filedata = get_file_from_url(url=url)
        # print(filedata)
        file = open(f"{issue_id}", "w")
        file.write(filedata)
        file.close()
    else:
        raise KeyError("wrong issue id")
        


In [ ]:
def scan_report_for_percentage(report_text):
    matches = re.findall(pattern=r"\(\d+\%\)", string=report_text)
    if matches:
      result = [match.strip(r"()%") for match in matches]
      result_int = [int(match) for match in result]
      # average = sum(result_int)/len(result_int)
      max_result = max(result_int)
      return max_result
    else:
      return 0

In [ ]:
def get_similarity_scores_top_x(data, language, input_issue_id, top_ranked_issue_ids):
    get_patch_file_from_issue_id(input_issue_id)
    moss_similarities = []
    for issue_id in top_ranked_issue_ids:
        get_patch_file_from_issue_id(issue_id)
        url = moss_get_comparison_report_url(language, f"{input_issue_id}", f"{issue_id}")
        print(f"report being scanned from {url}")
        report_data = get_file_from_url(url)
        moss_similarity = scan_report_for_percentage(report_text=report_data)
        moss_similarities.append(moss_similarity)
    return moss_similarities

In [ ]:
top_3_issues_list = [sim_list[0:3] for sim_list in sorted_similarity_lists]
top_3_issues_list_pd = pd.DataFrame(top_3_issues_list, index=preprocessed_issue_indices)
top_3_issues_list_pd
top_3_issues_list_pd.to_csv("exp_1_results_issue_ids.csv")

array([    0,     1,     2, ..., 12585, 12586, 12587])

In [ ]:
similarity_per_issue = []
for index in range(len(top_3_issues_list)):
  similarities = get_similarity_scores_top_x(df, "javascript", preprocessed_issue_indices[index], top_3_issues_list[index])
  similarity_per_issue.append(similarities)

similarity_per_issue

In [ ]:
exp1_results = pd.DataFrame(similarity_per_issue, index=preprocessed_issue_indices)

In [ ]:
exp1_results.to_csv("exp1_results.csv")

In [ ]:
df['patch_url'][3183]

'https://github.com/angular/angular/pull/22275.patch'

In [ ]:
df['patch_url'][0]

'https://github.com/angular/angular/pull/34111.patch'

In [ ]:
df['patch_url'][5616]

'https://github.com/angular/angular/pull/43529.patch'

For each row

1. first we set aside the issue text of open issue

2. then we provide ChatGPT the open issue's text and the most similar patch file and ask it for a solution

3. We compare the results using MOSS


In [ ]:
exp1_results

,0,1,2
0,99,12,0
1,96,0,14
2,67,1,3
3,99,8,10
4,99,1,6
5,99,11,9
6,99,53,9
7,99,0,0
8,99,0,0
9,99,0,7


In [ ]:
# df["patch_url"][6]
df["patch_url"][2280]

'https://github.com/angular/angular/pull/38741.patch'

In [ ]:
top_3_issues_list

[[0, 5616, 3183],
 [1, 3375, 1852],
 [2, 8298, 8357],
 [3, 1509, 2540],
 [4, 966, 4423],
 [5, 2058, 3652],
 [6, 2280, 4630],
 [7, 2041, 7643],
 [8, 3581, 5296],
 [9, 3175, 2474],
 [8555, 2301, 8424],
 [8556, 8702, 6410],
 [8557, 8581, 8632],
 [8558, 9721, 9992],
 [8559, 8811, 9440],
 [8560, 9009, 9671],
 [8561, 8586, 9007],
 [8562, 8581, 4608],
 [8563, 6845, 2357],
 [8564, 9786, 9245],
 [12000, 12125, 10280],
 [12001, 12519, 10674],
 [12002, 10712, 10994],
 [12003, 11194, 3432],
 [12004, 11922, 11528],
 [12005, 11789, 11334],
 [12006, 12158, 10586],
 [12007, 4726, 254],
 [12008, 11262, 8902],
 [12009, 12272, 7731]]

In [ ]:
t3issues = [[0, 5616, 3183],
 [1, 3375, 1852],
 [2, 8298, 8357],
 [3, 1509, 2540],
 [4, 966, 4423],
 [5, 2058, 3652],
 [6, 2280, 4630],
 [7, 2041, 7643],
 [8, 3581, 5296],
 [9, 3175, 2474],
 [8555, 2301, 8424],
 [8556, 8702, 6410],
 [8557, 8581, 8632],
 [8558, 9721, 9992],
 [8559, 8811, 9440],
 [8560, 9009, 9671],
 [8561, 8586, 9007],
 [8562, 8581, 4608],
 [8563, 6845, 2357],
 [8564, 9786, 9245],
 [12000, 12125, 10280],
 [12001, 12519, 10674],
 [12002, 10712, 10994],
 [12003, 11194, 3432],
 [12004, 11922, 11528],
 [12005, 11789, 11334],
 [12006, 12158, 10586],
 [12007, 4726, 254],
 [12008, 11262, 8902],
 [12009, 12272, 7731]]

In [ ]:
for index in range(len(t3issues)):
  print(f"{index} & {t3issues[index][0]} & {t3issues[index][1]} & {t3issues[index][2]}")

0 & 0 & 5616 & 3183
1 & 1 & 3375 & 1852
2 & 2 & 8298 & 8357
3 & 3 & 1509 & 2540
4 & 4 & 966 & 4423
5 & 5 & 2058 & 3652
6 & 6 & 2280 & 4630
7 & 7 & 2041 & 7643
8 & 8 & 3581 & 5296
9 & 9 & 3175 & 2474
10 & 8555 & 2301 & 8424
11 & 8556 & 8702 & 6410
12 & 8557 & 8581 & 8632
13 & 8558 & 9721 & 9992
14 & 8559 & 8811 & 9440
15 & 8560 & 9009 & 9671
16 & 8561 & 8586 & 9007
17 & 8562 & 8581 & 4608
18 & 8563 & 6845 & 2357
19 & 8564 & 9786 & 9245
20 & 12000 & 12125 & 10280
21 & 12001 & 12519 & 10674
22 & 12002 & 10712 & 10994
23 & 12003 & 11194 & 3432
24 & 12004 & 11922 & 11528
25 & 12005 & 11789 & 11334
26 & 12006 & 12158 & 10586
27 & 12007 & 4726 & 254
28 & 12008 & 11262 & 8902
29 & 12009 & 12272 & 7731


In [ ]:
get_patch_url_from_issue_id(5616)

https://github.com/angular/angular/pull/43529.patch


'https://github.com/angular/angular/pull/43529.patch'

In [ ]:
moss_get_comparison_report_url("javascript", "10280", "10586")

**
Report Url: http://moss.stanford.edu/results/9/6997739913381


'http://moss.stanford.edu/results/9/6997739913381'

In [ ]:
moss_get_comparison_report_url("javascript", issue_ind, f"{issue_ind}_comparison")

In [ ]:
print("open issue id : open issue patch url : closed issue id : close issue patch url")

for issues_list in top_3_issues_list:
  print(f" {issues_list[0]} : {get_patch_url_from_issue_id(issues_list[0])} : {issues_list[1]}: {get_patch_url_from_issue_id(issues_list[1])}")

open issue id : open issue patch url : closed issue id : close issue patch url
 0 : https://github.com/angular/angular/pull/34111.patch : 5616: https://github.com/angular/angular/pull/43529.patch
 1 : https://github.com/angular/angular/pull/34173.patch : 3375: https://github.com/angular/angular/pull/40324.patch
 2 : https://github.com/angular/angular/pull/34002.patch : 8298: https://github.com/angular/angular/pull/46051.patch
 3 : https://github.com/angular/angular/pull/34146.patch : 1509: https://github.com/angular/angular/pull/37420.patch
 4 : https://github.com/angular/angular/pull/34034.patch : 966: https://github.com/angular/angular/pull/36365.patch
 5 : https://github.com/angular/angular/pull/34073.patch : 2058: https://github.com/angular/angular/pull/38370.patch
 6 : https://github.com/angular/angular/pull/34122.patch : 2280: https://github.com/angular/angular/pull/38741.patch
 7 : https://github.com/angular/angular/pull/33218.patch : 2041: https://github.com/angular/angular/pul

In [ ]:
"12009_comparison"

In [ ]:
df["text"][8555]

'[WIP] Failing test user-blocking update warning bug Trying create failing test potential bug encountered "user blocking update" warning CODE. demo reproduces issue consistently. Source here: https://gist.github.com/aweary/6cd04d4e9475927cdad63cbaae9031a9 ### Steps Reproduce Go http://domineering-sack.surge.sh/ Wait initial suspend resolve Click "bar" button warning happen keep clicking back forth "foo" "bar" ### Results get warning: REFERENCE think expected, update causes suspend already inside CODE.'

In [ ]:
preprocessed_issue_indices

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 8555,
 8556,
 8557,
 8558,
 8559,
 8560,
 8561,
 8562,
 8563,
 8564,
 12000,
 12001,
 12002,
 12003,
 12004,
 12005,
 12006,
 12007,
 12008,
 12009]

In [ ]:
df["text"][8562 ]

'[Suspense] Failing test suspense config causes test fail think should. Issue: #16961'

In [ ]:
!unzip results.zip

Archive:  results.zip
caution: filename not matched:  ./


In [ ]:
import os
def list_files(folder_path):
    result = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            file_path = os.path.join(root, file)
            result.append(file_path)
    return result

In [ ]:
sorted_list = list_files("./results")
sorted_list.sort()
sorted_list

['./results/0_comparison',
 './results/12000_comparison',
 './results/12001_comparison',
 './results/12002_comparison',
 './results/12003_comparison',
 './results/12004_comparison',
 './results/12005_comparison',
 './results/12006_comparison',
 './results/12007_comparison',
 './results/12008_comparison',
 './results/12009_comparison',
 './results/1_comparison',
 './results/2_comparison',
 './results/3_comparison',
 './results/4_comparison',
 './results/5_comparison',
 './results/6_comparison',
 './results/7_comparison',
 './results/8555_comparison',
 './results/8556_comparison',
 './results/8557_comparison',
 './results/8558_comparison',
 './results/8559_comparison',
 './results/8560_comparison',
 './results/8561_comparison',
 './results/8563_comparison',
 './results/8662_comparison',
 './results/8664_comparison',
 './results/8_comparison',
 './results/9_comparison']

In [ ]:
comparison_result = []

for issue_ind in preprocessed_issue_indices:
  print(issue_ind)
  print(f"{issue_ind}_comparison")
  similarity = moss_get_comparison_report_url("javascript", f"{issue_ind}", f"{issue_ind}_comparison")
  comparison_result.append((issue_ind, similarity))

0
0_comparison
**
Report Url: http://moss.stanford.edu/results/5/2461366053643
1
1_comparison
**
Report Url: http://moss.stanford.edu/results/5/6632392407368
2
2_comparison
**
Report Url: http://moss.stanford.edu/results/7/1642294540974
3
3_comparison
**
Report Url: http://moss.stanford.edu/results/0/7550485256101
4
4_comparison
**
Report Url: http://moss.stanford.edu/results/7/5753908093060
5
5_comparison
**
Report Url: http://moss.stanford.edu/results/2/1463750824715
6
6_comparison
**
Report Url: http://moss.stanford.edu/results/4/2580191656920
7
7_comparison
**
Report Url: http://moss.stanford.edu/results/5/707300790037
8
8_comparison
**
Report Url: http://moss.stanford.edu/results/5/1213403794417
9
9_comparison
**
Report Url: http://moss.stanford.edu/results/4/5734103825847
8555
8555_comparison
**
Report Url: http://moss.stanford.edu/results/2/3937526662807
8556
8556_comparison
**
Report Url: http://moss.stanford.edu/results/7/2121145232553
8557
8557_comparison
**
Report Url: http:

In [ ]:
comparison_result

[(0, 'http://moss.stanford.edu/results/5/2461366053643'),
 (1, 'http://moss.stanford.edu/results/5/6632392407368'),
 (2, 'http://moss.stanford.edu/results/7/1642294540974'),
 (3, 'http://moss.stanford.edu/results/0/7550485256101'),
 (4, 'http://moss.stanford.edu/results/7/5753908093060'),
 (5, 'http://moss.stanford.edu/results/2/1463750824715'),
 (6, 'http://moss.stanford.edu/results/4/2580191656920'),
 (7, 'http://moss.stanford.edu/results/5/707300790037'),
 (8, 'http://moss.stanford.edu/results/5/1213403794417'),
 (9, 'http://moss.stanford.edu/results/4/5734103825847'),
 (8555, 'http://moss.stanford.edu/results/2/3937526662807'),
 (8556, 'http://moss.stanford.edu/results/7/2121145232553'),
 (8557, 'http://moss.stanford.edu/results/9/4396573087173'),
 (8558, 'http://moss.stanford.edu/results/4/9485692289631'),
 (8559, 'http://moss.stanford.edu/results/4/3118455150138'),
 (8560, 'http://moss.stanford.edu/results/4/9279230907942'),
 (8561, 'http://moss.stanford.edu/results/4/99645429812

In [ ]:
comparison_list_scores = [(x[0], scan_report_for_percentage(get_file_from_url(x[1]))) for x in comparison_result]

In [ ]:
comparison_list_scores

[(0, 0),
 (1, 61),
 (2, 0),
 (3, 0),
 (4, 0),
 (5, 0),
 (6, 0),
 (7, 0),
 (8, 0),
 (9, 0),
 (8555, 0),
 (8556, 0),
 (8557, 0),
 (8558, 0),
 (8559, 0),
 (8560, 0),
 (8561, 96),
 (8562, 0),
 (8563, 0),
 (8564, 99),
 (12000, 0),
 (12001, 0),
 (12002, 9),
 (12003, 72),
 (12004, 18),
 (12005, 28),
 (12006, 0),
 (12007, 45),
 (12008, 15),
 (12009, 99)]